Get community building samples from euss and renormalize the sample weight

In [11]:
import pandas as pd
from pathlib import Path
import getpass

user = getpass.getuser()
print(f"The user is {user}")

The user is lliu2


In [12]:
## Dwelling unit count by community 
Total_dwelling_unit_count = {
    "duluth": 39762,
} # TODO add others here

In [13]:
community = 'duluth' # TODO change community name

In [14]:
# set path
if user == "ylou2":
    euss_dir = 'data_/euss_res_final_2018_550k_20220901/' # TODO
elif user == "lliu2":
    euss_dir = "/Volumes/Lixi_Liu/euss_aws"
else:
    print(f"new user={user}, set euss_dir path here with an elif statement")

euss_dir = Path(euss_dir).resolve()
output_dir = Path(".").resolve() / "data_" / "community_building_samples" / community

print(f"euss_dir: {euss_dir}")
print(f"output_dir: {output_dir}")

euss_dir: /Volumes/Lixi_Liu/euss_aws
output_dir: /Users/lliu2/Documents/GitHub/ResStock/euss_cleap/data_/community_building_samples/duluth


In [15]:
building_id_weight = pd.read_csv(f'data_/downsampled_buildings_id/euss1_2018_results_up00__downsampled_method1__{community}.csv')
building_id_weight['building_id'] = building_id_weight['building_id'].astype(int)
building_id_weight = building_id_weight.set_index('building_id')
building_id_weight["sample_weight"] *= Total_dwelling_unit_count[community] / building_id_weight["sample_weight"].sum() # normalize 
assert building_id_weight["sample_weight"].sum() == Total_dwelling_unit_count[community], "renormalization failed"

print(f"building_id_weight for {community} has {len(building_id_weight)} rows")

for i in range(11):
    euss_up = pd.read_parquet(euss_dir / f'results_up{i:02d}.parquet', engine='pyarrow')
    euss_up["building_id"] = euss_up["building_id"].astype(int)
    community_up = euss_up.join(building_id_weight, on="building_id", how="right")
    n_downselected = len(community_up)

    community_up = community_up.loc[community_up["completed_status"] == "Success"].reset_index(drop=True) # drop failed sims
    community_up.to_parquet(output_dir / f"up{i:02d}.parquet")
    
    print(f" - for up{i:02d}, community_up has {n_downselected} rows after downselection, and {len(community_up)} rows after dropping failed sims")

print("all done!")


building_id_weight for duluth has 1217 rows
 - for up00, community_up has 1217 rows after downselection and 1217 rows after dropping failed sims
 - for up01, community_up has 1217 rows after downselection and 1048 rows after dropping failed sims
